In [1]:
import numpy as np
import random
import cv2
import os,sys
from PIL import Image, ImageDraw
import math
from random import shuffle

%matplotlib inline
ROOT_DIR = os.getcwd()

In [2]:
DR_IMAGE_DIR = os.path.join(ROOT_DIR, "GenerateData-GroundTruth/Sub1_OriginalImage") #DRDataset/SubChallenge_1_Images
    # Load a random image from the images folder
file_names = next(os.walk(DR_IMAGE_DIR))[2]

if ".DS_Store" in file_names:
    file_names.remove(".DS_Store")

In [3]:
def randomCoordinates(x = 4000, y = 2800):
    x = random.randint(280,x)  #把黑边空出去了
    y = random.randint(1,y)
    return x, y

def drawCircle(img, x = 800, y = 1000, radius = 25, R = 0, G = 0, B = 255):  # length = 2000, width = 1500
    image = cv2.circle(img,(x,y), radius, (B, G, R), -1)    
    return image

def compareMatrix(res_img, draw_img):
    return (res_img == draw_img).any()

def convertGrayImage(img):
    image = Image.fromarray(img)
    img_gray=np.array(image.convert("L")) 
    return img_gray

def generateMask(img, radius, x_coor, y_coor):
    [x, y, z] = img.shape
    imgZero = np.zeros([x, y, 3], dtype = np.uint8)
    img_draw = drawCircle(imgZero, x_coor, y_coor, radius, R = 1, G = 1, B = 1)
    img_draw = img_draw * img
    return img_draw

def classLesionResults(classresults, typeLesion):   #typeLesion 包括0(background), 1(MA), 2(HR), 3(HE), 4(SE)  
    classtempResults = []
    for i in range(len(classresults)):
        if classresults[i][0] == typeLesion:
            classtempResults.append(classresults[i])
    classtempResults = np.array(classtempResults)
    return classtempResults

def calculateTpFp(mat):
    mattemp = mat[:, 0] - mat[:, 1]
    TP = len(mattemp[mattemp == 0])
    FP = len(mattemp[mattemp != 0])
    return TP, FP

def calculateTnFn(mat):
    mattemp = mat[:, 0] - mat[:, 1]
    TN = len(mattemp[mattemp == 0])
    FN = len(mattemp[mattemp != 0])
    return TN, FN

def calculateSenSpecAcc(TP, FP, TN, FN):
    SEN = TP/(TP+FN)
    SPEC = TN/(TN+FP)
    ACC = (TP+TN)/(TP+FN+TN+FP)
    return SEN, SPEC, ACC

def randomBlackBlock(img, radius, num = 50):
    [x, y, z] = img.shape
    imgZeroCopy = np.zeros([x,y,z], dtype = np.uint8)
    imgZeroCopy = convertGrayImage(imgZeroCopy)
    centerlist = []
    for i in range(num):
        #print("i=", i)
        A = 1
        while A:
#             m, n = randomCoordinates()
#             #print(m, n)
#             imgZero = np.zeros([x,y,3], dtype = np.uint8)
#             img_draw = drawCircle(imgZero, m, n, radius, R = 1, G = 1, B = 1)
#             img_draw = img_draw * img
            m, n = randomCoordinates()
            img_draw = generateMask(img, radius, m, n)
            img_draw = convertGrayImage(img_draw)
            #cv2.imwrite('test/test_img_draw%02d.png'%(i), img1)
            #print((img == img_draw).all())
            if (imgZeroCopy == img_draw).all():
                centerlist.append([m, n, radius, 0])
                A = 0
    centerLocation = np.array(centerlist)
    return centerLocation

def calculateLesionCoordinates(img, typeLesion = 1):   #typeLesion 包括1(MA), 2(HR), 3(HE), 4(SE)
    img_gray = convertGrayImage(img)
    mask_arr = np.array(img_gray != 0).astype(np.uint8)
    mask_arr = mask_arr*255
    [x,y]=img_gray.shape
    Mask_arr= np.zeros([x,y,3], dtype = np.uint8)

    ROI,contours,hierarchy= cv2.findContours(mask_arr,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    #box = []
    centerlist = [] 
    mylist2 = []
    #for item in data: 
    
    lesionNum = len(contours)
    area = cv2.countNonZero(mask_arr)

    for i in range(len(contours)):
        x,y,w,h=cv2.boundingRect(contours[i]) #（x,y）为矩形左上角的坐标，（w,h）是矩形的宽和高
        centerX = round(x + w/2)
        centerY = round(y + h/2)
        centerlist.append([centerX, centerY, max(math.ceil(w/2), math.ceil(h/2)), typeLesion])
    
    centerLocation = np.array(centerlist)
    #print(centerLocation)
    return centerLocation

# 该函数用来判断病变区域的lesion level是否存在存在病变
def trueOrFalse(img_result, x_coor, y_coor, radius, typeLesion): #typeLesion 包括0(background), 1(MA), 2(HR), 3(HE), 4(SE)
    [x, y, z] = img_result.shape
    imgGe = np.ones([x,y,z], dtype = np.uint8)*125
    if typeLesion == 1:
        image = drawCircle(imgGe, x_coor, y_coor, radius, R = 0, G = 255, B = 255)
    elif typeLesion == 2:
        image = drawCircle(imgGe, x_coor, y_coor, radius, R = 0, G = 0, B = 255)
    elif typeLesion == 3:
        image = drawCircle(imgGe, x_coor, y_coor, radius, R = 255, G = 0, B = 255)
    elif typeLesion == 4:
        image = drawCircle(imgGe, x_coor, y_coor, radius, R = 255, G = 255, B = 0)
    elif typeLesion == 0:
        image = drawCircle(imgGe, x_coor, y_coor, radius, R = 0, G = 0, B = 0)
    else:
        print("trueOrFalse lose parameter")
    cv2.imwrite('test_img_draw2.png', image)
    image = convertGrayImage(image)
    
    img_result = generateMask(img_result, radius, x_coor, y_coor)
    imgGenerate = np.ones([x,y,z], dtype = np.uint8)*125
    imageCircle = drawCircle(imgGenerate, x_coor, y_coor, radius, R = 0, G = 0, B = 0)
    img_result = img_result + imageCircle  #img_result是结果图， imageCircle是遮罩图，目标区域是黑色，其它区域RGB值为（125，125，125）
    # cv2.imwrite('test_src111.png', img_result)
    img_result = convertGrayImage(img_result)
    
    if typeLesion == 0:
        return (image == img_result).all()
    else:
        return (image == img_result).any()

In [4]:
for i in range(len(file_names)):  #len(file_names)
    newfile_name = file_names[i].strip('.jpg')
    filepath_ma = "GenerateData-GroundTruth/MA/MA_ANN/"+newfile_name+"_MA.png"
    filepath_hr = "GenerateData-GroundTruth/HR/HR_ANN/"+newfile_name+"_HR.png"
    filepath_he = "GenerateData-GroundTruth/HE/HE_ANN/"+newfile_name+"_HE.png"
    filepath_se = "GenerateData-GroundTruth/SE/SE_ANN/"+newfile_name+"_SE.png"

    resultspath_ma = "GenerateData-GroundTruth/Results/MA_1010/"+newfile_name+".png"
    resultspath_hr = "GenerateData-GroundTruth/Results/HR_2002/"+newfile_name+".png"
    resultspath_he = "GenerateData-GroundTruth/Results/HE_1030/"+newfile_name+".png"
    resultspath_se = "GenerateData-GroundTruth/Results/SE_1004/"+newfile_name+".png"

    src_ma = cv2.imread(filepath_ma)
    src_hr = cv2.imread(filepath_hr)
    src_he = cv2.imread(filepath_he)
    src_se = cv2.imread(filepath_se)
    src = src_ma + src_hr + src_he + src_se
    #src_ma_gray = cv2.imread(filepath_ma, cv2.COLOR_BGR2RGB)
    #cv2.imwrite('test_src.png', src)

    results_ma = cv2.imread(resultspath_ma)
    results_hr = cv2.imread(resultspath_hr)
    results_he = cv2.imread(resultspath_he)
    results_se = cv2.imread(resultspath_se)

    BlackLocation = randomBlackBlock(src, 25, 50)  
    MaLesionLocation = calculateLesionCoordinates(results_ma, 1)
    HrLesionLocation = calculateLesionCoordinates(results_hr, 2)
    HeLesionLocation = calculateLesionCoordinates(results_he, 3)
    SeLesionLocation = calculateLesionCoordinates(results_se, 4)
    CenterLocation = np.concatenate((BlackLocation, MaLesionLocation, HrLesionLocation, HeLesionLocation, SeLesionLocation), axis = 0)

    # print(len(CenterLocation))
    # print(CenterLocation)

    # 一幅图像统计lesion level结果

    ClassResultsList = [] 
    for i in range(len(CenterLocation)):
        if trueOrFalse(src, CenterLocation[i][0], CenterLocation[i][1], CenterLocation[i][2], CenterLocation[i][3]):
            ClassResultsList.append([CenterLocation[i][3], round(random.uniform(0.89, 1), 2)])
        else:
            ClassResultsList.append([5, round(random.uniform(0.89, 1), 2)])

    np.set_printoptions(suppress=True)
    classresultslist = np.array(ClassResultsList)
    #print(classresultslist)
    resultsClassScore = np.column_stack((CenterLocation, classresultslist))
    print(resultsClassScore)
    # print(len(resultsClassScore))

    classresultsall = resultsClassScore[:, 3:5]
    #print(resultsClassScore[:, 3:5])

    classBgResults = classLesionResults(classresultsall, 0)
    classMaResults = classLesionResults(classresultsall, 1)
    classHrResults = classLesionResults(classresultsall, 2)
    classHeResults = classLesionResults(classresultsall, 3)
    classSeResults = classLesionResults(classresultsall, 4)

    TN, FN = calculateTnFn(classBgResults)
    TP_Ma, FP_Ma = calculateTpFp(classMaResults)
    TP_Hr, FP_Hr = calculateTpFp(classHrResults)
    TP_He, FP_He = calculateTpFp(classHeResults)
    TP_Se, FP_Se = calculateTpFp(classSeResults)

    SenMa, SpecMa, AccMa = calculateSenSpecAcc(TP_Ma, FP_Ma, TN, FN)
    SenHr, SpecHr, AccHr = calculateSenSpecAcc(TP_Hr, FP_Hr, TN, FN)
    SenHe, SpecHe, AccHe = calculateSenSpecAcc(TP_He, FP_He, TN, FN)
    SenSe, SpecSe, AccSe = calculateSenSpecAcc(TP_Se, FP_Se, TN, FN)

    print(SenMa, SpecMa, AccMa)
    print(SenHr, SpecHr, AccHr)
    print(SenHe, SpecHe, AccHe)
    print(SenSe, SpecSe, AccSe)

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [7]:
trueOrFalse(src, MaLesionLocation[0][0], MaLesionLocation[0][1], MaLesionLocation[0][2],1)
trueOrFalse(src, BlackLocation[1][0], BlackLocation[1][1], BlackLocation[1][2],0)
trueOrFalse(src, MaLesionLocation[0][0], MaLesionLocation[0][1], MaLesionLocation[0][2],0)
trueOrFalse(src, 1852, 145, 50,0)

True

True

False

True

In [ ]:
for i in range(len(file_names)):  #len(file_names)
    OriginalFilePath = DR_IMAGE_DIR+"/"+file_names[i]
    
    newfile_name = file_names[i].strip('.jpg')
    
    filepath_ma = "GenerateData-GroundTruth/MA/MA_ANN/"+newfile_name+"_MA.png"
    filepath_hr = "GenerateData-GroundTruth/HR/HR_ANN/"+newfile_name+"_HR.png"
    filepath_he = "GenerateData-GroundTruth/HE/HE_ANN/"+newfile_name+"_HE.png"
    filepath_se = "GenerateData-GroundTruth/SE/SE_ANN/"+newfile_name+"_SE.png"

    def existfile(groundfilepath, originalfilepath):
        if os.path.exists(groundfilepath):
            src = cv2.imread(groundfilepath)
        else:
            image_original = cv2.imread(originalfilepath)
            [x, y, z] = image_original.shape
            src = np.zeros([x, y, z], dtype = np.uint8)
        return src

    src_ma = existfile(filepath_ma, OriginalFilePath)
    src_hr = existfile(filepath_hr, OriginalFilePath)
    src_he = existfile(filepath_he, OriginalFilePath)
    src_se = existfile(filepath_se, OriginalFilePath)
    src = src_ma + src_hr + src_he + src_se

    BlackLocation = randomBlackBlock(src, 25, 50) 

    resultspath_ma = "GenerateData-GroundTruth/Results/MA_1010/"+newfile_name+".png"
    resultspath_hr = "GenerateData-GroundTruth/Results/HR_2002/"+newfile_name+".png"
    resultspath_he = "GenerateData-GroundTruth/Results/HE_1030/"+newfile_name+".png"
    resultspath_se = "GenerateData-GroundTruth/Results/SE_1004/"+newfile_name+".png"

    def SenSpecAcc(resultsfilepath, blacklocation, typeLesion):
        resultsfile = cv2.imread(resultsfilepath)
        LesionLocation = calculateLesionCoordinates(resultsfile, typeLesion)
        CenterLocation = np.concatenate((blacklocation, LesionLocation), axis = 0)

        # 一幅图像统计lesion level结果
        ClassResultsList = [] 
        for i in range(len(CenterLocation)):
            if trueOrFalse(src, CenterLocation[i][0], CenterLocation[i][1], CenterLocation[i][2], CenterLocation[i][3]):
                ClassResultsList.append([CenterLocation[i][3], round(random.uniform(0.89, 1), 2)])
            else:
                ClassResultsList.append([5, round(random.uniform(0.89, 1), 2)])

        np.set_printoptions(suppress=True)
        classresultslist = np.array(ClassResultsList)
        
        # print(classresultslist)
        resultsClassScore = np.column_stack((CenterLocation, classresultslist))
        # print(resultsClassScore)
        # print(len(resultsClassScore))

        classresultsall = resultsClassScore[:, 3:5]
        # print(resultsClassScore[:, 3:5])

        classBgResults = classLesionResults(classresultsall, 0)
        classResults = classLesionResults(classresultsall, typeLesion)

        TN, FN = calculateTnFn(classBgResults)
        TP, FP = calculateTpFp(classResults)

        Sen, Spec, Acc = calculateSenSpecAcc(TP, FP, TN, FN)

        return Sen, Spec, Acc

    print("图像 "+file_names[i]+" 的Sen, Spec, Acc分别是：")
    
    if os.path.exists(filepath_ma):
        SenMa, SpecMa, AccMa = SenSpecAcc(resultspath_ma, BlackLocation, 1)
        print("SenMa, SpecMa, AccMa =", SenMa, SpecMa, AccMa)
    
    if os.path.exists(filepath_hr):
        SenHr, SpecHr, AccHr = SenSpecAcc(resultspath_hr, BlackLocation, 2)
        print("SenHr, SpecHr, AccHr =", SenHr, SpecHr, AccHr)
        
    if os.path.exists(filepath_he):
        SenHe, SpecHe, AccHe = SenSpecAcc(resultspath_he, BlackLocation, 2)
        print("SenHe, SpecHe, AccHe =", SenHe, SpecHe, AccHe)
        
    if os.path.exists(filepath_se):
        SenSe, SpecSe, AccSe = SenSpecAcc(resultspath_se, BlackLocation, 2)
        print("SenSe, SpecSe, AccSe =", SenSe, SpecSe, AccSe)
    
#     resultspath_ma = "GenerateData-GroundTruth/Results/MA_1010/"+newfile_name+".png"
#     resultspath_hr = "GenerateData-GroundTruth/Results/HR_2002/"+newfile_name+".png"
#     resultspath_he = "GenerateData-GroundTruth/Results/HE_1030/"+newfile_name+".png"
#     resultspath_se = "GenerateData-GroundTruth/Results/SE_1004/"+newfile_name+".png"

#     src_ma = cv2.imread(filepath_ma)
#     src_hr = cv2.imread(filepath_hr)
#     src_he = cv2.imread(filepath_he)
#     src_se = cv2.imread(filepath_se)
#     src = src_ma + src_hr + src_he + src_se
#     #src_ma_gray = cv2.imread(filepath_ma, cv2.COLOR_BGR2RGB)
#     #cv2.imwrite('test_src.png', src)

#     results_ma = cv2.imread(resultspath_ma)
#     results_hr = cv2.imread(resultspath_hr)
#     results_he = cv2.imread(resultspath_he)
#     results_se = cv2.imread(resultspath_se)

#     BlackLocation = randomBlackBlock(src, 25, 50)  
#     MaLesionLocation = calculateLesionCoordinates(results_ma, 1)
#     HrLesionLocation = calculateLesionCoordinates(results_hr, 2)
#     HeLesionLocation = calculateLesionCoordinates(results_he, 3)
#     SeLesionLocation = calculateLesionCoordinates(results_se, 4)
#     CenterLocation = np.concatenate((BlackLocation, MaLesionLocation, HrLesionLocation, HeLesionLocation, SeLesionLocation), axis = 0)

#     # print(len(CenterLocation))
#     # print(CenterLocation)

#     # 一幅图像统计lesion level结果

#     ClassResultsList = [] 
#     for i in range(len(CenterLocation)):
#         if trueOrFalse(src, CenterLocation[i][0], CenterLocation[i][1], CenterLocation[i][2], CenterLocation[i][3]):
#             ClassResultsList.append([CenterLocation[i][3], round(random.uniform(0.89, 1), 2)])
#         else:
#             ClassResultsList.append([5, round(random.uniform(0.89, 1), 2)])

#     np.set_printoptions(suppress=True)
#     classresultslist = np.array(ClassResultsList)
#     #print(classresultslist)
#     resultsClassScore = np.column_stack((CenterLocation, classresultslist))
#     print(resultsClassScore)
#     # print(len(resultsClassScore))

#     classresultsall = resultsClassScore[:, 3:5]
#     #print(resultsClassScore[:, 3:5])

#     classBgResults = classLesionResults(classresultsall, 0)
#     classMaResults = classLesionResults(classresultsall, 1)
#     classHrResults = classLesionResults(classresultsall, 2)
#     classHeResults = classLesionResults(classresultsall, 3)
#     classSeResults = classLesionResults(classresultsall, 4)

#     TN, FN = calculateTnFn(classBgResults)
#     TP_Ma, FP_Ma = calculateTpFp(classMaResults)
#     TP_Hr, FP_Hr = calculateTpFp(classHrResults)
#     TP_He, FP_He = calculateTpFp(classHeResults)
#     TP_Se, FP_Se = calculateTpFp(classSeResults)

#     SenMa, SpecMa, AccMa = calculateSenSpecAcc(TP_Ma, FP_Ma, TN, FN)
#     SenHr, SpecHr, AccHr = calculateSenSpecAcc(TP_Hr, FP_Hr, TN, FN)
#     SenHe, SpecHe, AccHe = calculateSenSpecAcc(TP_He, FP_He, TN, FN)
#     SenSe, SpecSe, AccSe = calculateSenSpecAcc(TP_Se, FP_Se, TN, FN)

#     print(SenMa, SpecMa, AccMa)
#     print(SenHr, SpecHr, AccHr)
#     print(SenHe, SpecHe, AccHe)
#     print(SenSe, SpecSe, AccSe)

[[ 1207.      51.      25.       0.       0.       0.96]
 [ 3935.     954.      25.       0.       0.       0.96]
 [ 2510.    1520.      25.       0.       0.       0.96]
 [ 1007.    2464.      25.       0.       0.       0.97]
 [ 3957.     683.      25.       0.       0.       0.93]
 [ 3243.    1333.      25.       0.       0.       0.93]
 [ 1315.     553.      25.       0.       0.       0.99]
 [  313.    1002.      25.       0.       0.       0.9 ]
 [  467.    2681.      25.       0.       0.       0.97]
 [ 2610.    2478.      25.       0.       0.       0.91]
 [  821.    1185.      25.       0.       0.       0.99]
 [  514.    1122.      25.       0.       0.       0.99]
 [ 1476.    2200.      25.       0.       0.       0.9 ]
 [ 3380.    1058.      25.       0.       0.       0.96]
 [ 2031.    1300.      25.       0.       0.       0.93]
 [ 2664.    2433.      25.       0.       0.       0.94]
 [ 2305.    1887.      25.       0.       0.       0.99]
 [ 3567.     151.      25.     

[[ 1207.      51.      25.       0.       0.       0.99]
 [ 3935.     954.      25.       0.       0.       1.  ]
 [ 2510.    1520.      25.       0.       0.       0.93]
 [ 1007.    2464.      25.       0.       0.       0.96]
 [ 3957.     683.      25.       0.       0.       0.95]
 [ 3243.    1333.      25.       0.       0.       0.94]
 [ 1315.     553.      25.       0.       0.       0.98]
 [  313.    1002.      25.       0.       0.       0.93]
 [  467.    2681.      25.       0.       0.       0.98]
 [ 2610.    2478.      25.       0.       0.       0.92]
 [  821.    1185.      25.       0.       0.       0.96]
 [  514.    1122.      25.       0.       0.       0.95]
 [ 1476.    2200.      25.       0.       0.       1.  ]
 [ 3380.    1058.      25.       0.       0.       0.9 ]
 [ 2031.    1300.      25.       0.       0.       0.97]
 [ 2664.    2433.      25.       0.       0.       0.97]
 [ 2305.    1887.      25.       0.       0.       0.94]
 [ 3567.     151.      25.     

[[ 3395.       1.      25.       0.       0.       0.98]
 [ 1896.    1047.      25.       0.       0.       0.9 ]
 [ 1429.    2411.      25.       0.       0.       0.91]
 [ 3034.    2564.      25.       0.       0.       0.95]
 [ 2466.    1733.      25.       0.       0.       0.93]
 [ 2532.      10.      25.       0.       0.       0.95]
 [ 2981.    1721.      25.       0.       0.       0.9 ]
 [  562.    1553.      25.       0.       0.       0.98]
 [ 2329.     437.      25.       0.       0.       0.93]
 [ 1230.      25.      25.       0.       0.       1.  ]
 [ 1335.     609.      25.       0.       0.       0.97]
 [ 3990.     132.      25.       0.       0.       0.9 ]
 [ 1108.    2721.      25.       0.       0.       0.9 ]
 [  744.    1556.      25.       0.       0.       0.95]
 [ 1503.    2285.      25.       0.       0.       0.92]
 [ 2408.    2429.      25.       0.       0.       0.99]
 [ 3247.     272.      25.       0.       0.       0.95]
 [ 3230.     463.      25.     

[[ 3395.       1.      25.       0.       0.       0.93]
 [ 1896.    1047.      25.       0.       0.       0.99]
 [ 1429.    2411.      25.       0.       0.       0.91]
 [ 3034.    2564.      25.       0.       0.       0.98]
 [ 2466.    1733.      25.       0.       0.       0.9 ]
 [ 2532.      10.      25.       0.       0.       0.94]
 [ 2981.    1721.      25.       0.       0.       0.99]
 [  562.    1553.      25.       0.       0.       0.94]
 [ 2329.     437.      25.       0.       0.       0.89]
 [ 1230.      25.      25.       0.       0.       0.94]
 [ 1335.     609.      25.       0.       0.       0.93]
 [ 3990.     132.      25.       0.       0.       0.96]
 [ 1108.    2721.      25.       0.       0.       0.95]
 [  744.    1556.      25.       0.       0.       0.95]
 [ 1503.    2285.      25.       0.       0.       0.96]
 [ 2408.    2429.      25.       0.       0.       0.9 ]
 [ 3247.     272.      25.       0.       0.       0.9 ]
 [ 3230.     463.      25.     

[[ 3395.       1.      25.       0.       0.       0.95]
 [ 1896.    1047.      25.       0.       0.       1.  ]
 [ 1429.    2411.      25.       0.       0.       0.97]
 [ 3034.    2564.      25.       0.       0.       1.  ]
 [ 2466.    1733.      25.       0.       0.       0.97]
 [ 2532.      10.      25.       0.       0.       0.99]
 [ 2981.    1721.      25.       0.       0.       0.9 ]
 [  562.    1553.      25.       0.       0.       0.95]
 [ 2329.     437.      25.       0.       0.       0.96]
 [ 1230.      25.      25.       0.       0.       0.89]
 [ 1335.     609.      25.       0.       0.       0.94]
 [ 3990.     132.      25.       0.       0.       0.97]
 [ 1108.    2721.      25.       0.       0.       0.98]
 [  744.    1556.      25.       0.       0.       0.98]
 [ 1503.    2285.      25.       0.       0.       0.95]
 [ 2408.    2429.      25.       0.       0.       0.92]
 [ 3247.     272.      25.       0.       0.       0.95]
 [ 3230.     463.      25.     

In [27]:
print(DR_IMAGE_DIR+"/"+file_names[1])

/Users/victor/code4suixiaodan/Mask_RCNN-master/code4work/GenerateData-GroundTruth/Sub1_OriginalImage/IDRiD_33.jpg
